In [ ]:
#--------------------
using PowerModels
using Ipopt
using JuMP

path = "../ModelData/"
key = "case_ieee123_storage_"
file_ext = ".m"


# Instancate a Solver
#--------------------

nlp_solver = IpoptSolver(print_level=0)
# note: print_level changes the amount of solver information printed to the terminal


# Load System Data
# ----------------


periods = 0
for (root, dirs, files) in walkdir(path)
    for file in files
        if occursin(key, file)
            println(file)
            global periods=periods+1
        end
    end
end

#mp_data=[]

for i=1:55
    if i == 1
        data_1 = PowerModels.parse_file(string(path,key,i,file_ext))

        global mp_data = Dict{String,Any}(
            "name" => "$(data_1["name"])",
            "multinetwork" => true,
            "per_unit" => data_1["per_unit"],
            "baseMVA" => data_1["baseMVA"],
            "nw" => Dict{String,Any}()
        )

        delete!(data_1, "multinetwork")
        delete!(data_1, "per_unit")
        delete!(data_1, "baseMVA")
        global mp_data["nw"]["$(i)"] = data_1

    else
        data_2 = PowerModels.parse_file(string(path,key,i,file_ext))
        delete!(data_2, "multinetwork")
        delete!(data_2, "per_unit")
        delete!(data_2, "baseMVA")
        global mp_data["nw"]["$(i)"] = data_2
    end
end

PowerModels.standardize_cost_terms(mp_data, order=2)





In [2]:


horizon = 50

#for k=1:length(collect(keys(base[:nw])))#-horizon
for k=1:length(keys(mp_data["nw"]))-horizon
    global horizon_data = Dict{String,Any}(
            "name" => "Period $(k)",
            "multinetwork" => true,
            "per_unit" => mp_data["per_unit"],
            "baseMVA" => mp_data["baseMVA"],
            "nw" =>  Dict{String,Any}()
        )
    
    for w=k:k+horizon -1
        horizon_data["nw"][string(w)] = mp_data["nw"][string(w)]
    end
    
    global ref = PowerModels.build_ref(horizon_data)
    
    #planning_data = AC_OPF_MP_Storage(ref, k, horizon)
    #
    #horizon_data[:nw][k+1][]
    
    #println(planning_data[:nw][2][:gen][1]["ramp_agc"])
end

In [18]:
println(horizon_data["nw"]["7"]["storage"]["1"]["energy"])

0.0
